#Thinkful Data Science Course
##Unit 4: Predicting the Future; Lesson 1: Making Predictions
##Metrics and Cross-Validation

1) Take the loan data you used previously to build your linear regression model.

2) Break the data-set into 10 segments following the example provided here in KFold (http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators).

3) Compute each of the performance metric (MAE, MSE or R2) for all the folds. The average would be the performance of your model.

4) Comment on each of the performance metric you obtained.

###1) Take the loan data you used previously to build your linear regression model.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

# Import data
loansData = pd.read_csv('https://spark-public.s3.amazonaws.com/dataanalysis/loansData.csv')

In [2]:
# Split into numbers - will return in form [###, ###]
cleanFICORange = loansData['FICO.Range'].map(lambda x: x.split('-'))

In [3]:
# We have a string inside a list. Need to convert each string to integer
# To do this, we use a list comprehension
# We could either choose the first element, last element, or an average value
# The "[0]" at the end makes it so that we only choose the first element
cleanFICOScore = cleanFICORange.map(lambda x: [int(n) for n in x][0])

In [4]:
# According to Thinkful, to understand better, check out: http://carlgroner.me/Python/2011/11/09/An-Introduction-to-List-Comprehensions-in-Python.html

# Assign cleaned score to new column called "FICO.Score"
loansData['FICO.Score'] = cleanFICOScore

In [5]:
# CLEAN INTEREST RATE DATA
clean_ir = loansData["Interest.Rate"].map(lambda x: round(float(x.rstrip("%"))/100, 4))
loansData["Interest.Rate"] = clean_ir

In [6]:
# CLEAN LOAN LENGTH DATA
clean_loanLength = loansData["Loan.Length"].map(lambda x: float(x.rstrip("months")))
loansData["Loan.Length"] = clean_loanLength

####Linear Regression Model

In [7]:
# Linear regression model: 
# InterestRate = b + a1(FICOScore) + a2(LoanAmount)

intrate = loansData['Interest.Rate']
loanamt = loansData['Amount.Requested']
fico = loansData['FICO.Score']

# Create y and x variables
# The dependent variable
y = np.matrix(intrate).transpose()
# The independent variables shaped as columns
x1 = np.matrix(fico).transpose()
x2 = np.matrix(loanamt).transpose()
# put the two columns together to create an input matrix 
x = np.column_stack([x1,x2])

# Create linear model
X = sm.add_constant(x)
model = sm.OLS(y,X)
f = model.fit()

In [8]:
coeff = f.params

In [9]:
# plot:
line=[]
line2 = []
for j in fico:
	line.append(coeff[0] + coeff[1]*j + coeff[2]*10000)
	line2.append(coeff[0] + coeff[1]*j + coeff[2]*30000)

plt.close()
plt.scatter(fico,intrate)
plt.hold(True)
plt.plot(fico, line, label = '$10,000 Requested', color = 'blue')
plt.plot(fico, line2, label = '$30,000 Requested', color = 'green')
plt.legend(loc = 'upper right')
plt.ylabel('Interest Rate in %')
plt.xlabel('FICO Score')
plt.savefig('Fico_Scatter_10000&30000.png')

# Load to new CSV file
# loansData.to_csv('loansData_clean.csv', header=True, index=False)

###2) Break the data-set into 10 segments following the example provided here in KFold

In [10]:
len(fico)

2500

In [11]:
len(line)

2500

In [14]:
from sklearn import datasets, linear_model, cross_validation, grid_search
kf = cross_validation.KFold(len(fico), n_folds=10)

###3) Compute each of the performance metric (MAE, MSE or R2) for all the folds. The average would be the performance of your model.

In [24]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [16]:
for train_index, test_index in kf:
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lr = linear_model.LogisticRegression()
    for train_indices, test_indices in kf:
        lr.fit(x[train_indices], y[train_indices]).score(x[test_indices],y[test_indices])
#     mse = mean_squared_error(y_test, y_train)
#     mae = mean_absolute_error(y_test, y_train)
#     r2 = r2_score(y_test, y_train)
    
    
# >>> from sklearn.metrics import r2_score
# >>> y_true = [3, -0.5, 2, 7]
# >>> y_pred = [2.5, 0.0, 2, 8]
# >>> r2_score(y_true, y_pred)  
    


TRAIN: [ 250  251  252 ..., 2497 2498 2499] TEST: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 

/Users/molliepettit/anaconda/lib/python3.4/site-packages/sklearn/utils/validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: continuous is not supported

Example of K-Fold from https://www.quantstart.com/articles/Using-Cross-Validation-to-Optimise-a-Machine-Learning-Method-The-Regression-Setting

In [ ]:
def k_fold_cross_val_poly(folds, degrees, X, y):
    n = len(X)
    kf = KFold(n, n_folds=folds)
    kf_dict = dict([("fold_%s" % i,[]) for i in range(1, folds+1)])
    fold = 0
    for train_index, test_index in kf:
        fold += 1
        print "Fold: %s" % fold
        X_train, X_test = X.ix[train_index], X.ix[test_index]
        y_train, y_test = y.ix[train_index], y.ix[test_index]
        # Increase degree of linear regression polynomial order
        for d in range(1, degrees+1):
            print "Degree: %s" % d
            # Create the model and fit it
            polynomial_features = PolynomialFeatures(
                degree=d, include_bias=False
            )
            linear_regression = LinearRegression()
            model = Pipeline([
                ("polynomial_features", polynomial_features),
                ("linear_regression", linear_regression)
            ])
            model.fit(X_train, y_train)
            # Calculate the test MSE and append to the
            # dictionary of all test curves
            y_pred = model.predict(X_test)
            test_mse = mean_squared_error(y_test, y_pred)
            kf_dict["fold_%s" % fold].append(test_mse)
        # Convert these lists into numpy arrays to perform averaging
        kf_dict["fold_%s" % fold] = np.array(kf_dict["fold_%s" % fold])
    # Create the "average test MSE" series by averaging the 
    # test MSE for each degree of the linear regression model,
    # across each of the k folds.
    kf_dict["avg"] = np.zeros(degrees)
    for i in range(1, folds+1):
        kf_dict["avg"] += kf_dict["fold_%s" % i]
    kf_dict["avg"] /= float(folds)
    return kf_dict
..

###4) Comment on each of the performance metric you obtained.